In [1]:
import pandas as pd

import pyarrow.parquet as pq
from pyarrow.parquet import ParquetFile
import pyarrow as pa 
from sklearn.utils import resample
from imblearn.under_sampling import RandomUnderSampler
from imblearn.under_sampling import TomekLinks

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import datetime

In [2]:
parquet_file = pq.ParquetFile('../../input_data/test.parquet')

batch_size = 115_000_000
rows = next(parquet_file.iter_batches(batch_size=batch_size)) 
test = rows.to_pandas()  
rows=0
test

,user_id,adv_campaign_id,platform_id,adv_creative_id,event_date,banner_code,is_main
0,2714742,3026,2,4056,2024-09-23,8,True
1,2714742,2994,2,3954,2024-09-23,8,True
2,2714742,97,2,1752,2024-09-23,8,True
3,2714742,3539,2,1244,2024-09-23,8,True
4,2714742,2756,2,2003,2024-09-23,8,True
...,...,...,...,...,...,...,...
1983282,2398626,1099,2,3030,2024-09-23,8,True
1983283,2398626,488,2,1324,2024-09-23,8,True
1983284,2398626,49,2,1514,2024-09-23,8,True
1983285,2398626,1053,2,4211,2024-09-23,5,False


In [3]:
idx = (test.user_id.unique())
idx

array([2714742,  410633, 1201115, ..., 3207464,  790984, 2398626],
      dtype=int64)

In [4]:
len(test.user_id.unique())

159470

# формируем выборку только по юзерам, встречающимся в тесте, для них берем все лайки и рандомно отбираем часть игноров

In [5]:
parquet_file = pq.ParquetFile('../../input_data/train.parquet')

batch_size = 115_000_000
rows = next(parquet_file.iter_batches(batch_size=batch_size)) 
df = rows.to_pandas()  
rows=0
df

,user_id,adv_campaign_id,platform_id,adv_creative_id,event_date,banner_code,is_main,target
0,2853707,3352,3,3075,2024-09-17,6,True,0
1,2537244,4029,2,3260,2024-09-17,8,True,0
2,63033,1578,3,1109,2024-09-17,6,True,0
3,164702,3434,1,1079,2024-09-17,7,True,0
4,2802905,2208,3,3576,2024-09-17,6,True,0
...,...,...,...,...,...,...,...,...
114741030,2670849,3904,2,3275,2024-09-08,8,True,0
114741031,3187004,2142,2,238,2024-09-08,8,True,0
114741032,821310,2414,2,2654,2024-09-08,8,True,0
114741033,2282258,3120,2,551,2024-09-08,5,False,1


In [6]:
df = df[df.user_id.isin(idx)]
df

,user_id,adv_campaign_id,platform_id,adv_creative_id,event_date,banner_code,is_main,target
97462318,1555200,2226,2,1764,2024-09-17,8,True,0
97462319,2589012,3113,2,4223,2024-09-17,8,True,0
97462320,3192789,3366,3,2360,2024-09-17,6,True,0
97462321,29114,1996,2,413,2024-09-17,8,True,0
97462322,2243477,3095,2,595,2024-09-17,8,True,0
...,...,...,...,...,...,...,...,...
114741030,2670849,3904,2,3275,2024-09-08,8,True,0
114741031,3187004,2142,2,238,2024-09-08,8,True,0
114741032,821310,2414,2,2654,2024-09-08,8,True,0
114741033,2282258,3120,2,551,2024-09-08,5,False,1


In [7]:
df.target.value_counts()

target
0    16599655
1      154657
Name: count, dtype: int64

In [9]:
df1 = df[df.target==1]
df0 = df[df.target==0]
df0_ = df0.sample(frac = 0.7, random_state=42)
df1_ = pd.concat([df1, df0_], axis=0)
df1_

,user_id,adv_campaign_id,platform_id,adv_creative_id,event_date,banner_code,is_main,target
97462329,2491543,1277,2,4221,2024-09-17,8,True,1
97462336,1290433,3857,2,314,2024-09-17,5,False,1
97462407,2270457,4103,2,946,2024-09-17,8,True,1
97462778,513236,1042,2,368,2024-09-17,8,True,1
97462880,3054750,2900,2,1004,2024-09-17,8,True,1
...,...,...,...,...,...,...,...,...
109801267,3071169,3764,2,4190,2024-09-20,8,True,0
109735736,1752785,1420,2,2477,2024-09-20,8,True,0
97782852,2851171,2229,2,1006,2024-09-17,8,True,0
98208053,1217370,3762,1,2306,2024-09-17,7,True,0


In [10]:
len(df1_.user_id.unique())

158600

In [11]:
tl = TomekLinks(sampling_strategy='majority')
X, y = tl.fit_resample(df1_.drop(columns=['target', 'event_date']), df1_.target)

In [12]:
date = df1_.event_date.iloc[tl.sample_indices_].reset_index().drop(columns='index')
date

,event_date
0,2024-09-17
1,2024-09-17
2,2024-09-17
3,2024-09-17
4,2024-09-17
...,...
11741730,2024-09-20
11741731,2024-09-20
11741732,2024-09-17
11741733,2024-09-17


In [13]:
X2 = pd.concat([X, date], axis=1) 
X2 = pd.concat([X2, y], axis=1)        
X2

,user_id,adv_campaign_id,platform_id,adv_creative_id,banner_code,is_main,event_date,target
0,2491543,1277,2,4221,8,True,2024-09-17,1
1,1290433,3857,2,314,5,False,2024-09-17,1
2,2270457,4103,2,946,8,True,2024-09-17,1
3,513236,1042,2,368,8,True,2024-09-17,1
4,3054750,2900,2,1004,8,True,2024-09-17,1
...,...,...,...,...,...,...,...,...
11741730,3071169,3764,2,4190,8,True,2024-09-20,0
11741731,1752785,1420,2,2477,8,True,2024-09-20,0
11741732,2851171,2229,2,1006,8,True,2024-09-17,0
11741733,1217370,3762,1,2306,7,True,2024-09-17,0


In [14]:
X2.to_csv('all_test_users_f.csv', sep=',', index=False)